In [39]:
# squad2_path = "/Volumes/Share/tran_s2/SQuAD2_new/data/shortcut-study/train-squad-v2-adversarial-50unanswerable.json"
# adversarial_path = "/Volumes/Share/tran_s2/Frozen_Experiment/parallelQuAD/Data/PQuAD-attack/dev_pquad-attack.json"
squad2_path = "/Volumes/Share/tran_s2/Public_Datasets/SQuAD/squad2/train-v2.0-new-format.json"
adversarial_path = "/Volumes/Share/tran_s2/Public_Datasets/SQuAD/attack-raw/train-v2.0-adversarial-attack.json"

In [40]:
import json
with open(adversarial_path) as dataset_file:
    adversarial = json.load(dataset_file)
with open(squad2_path) as dataset_file:
    squad2 = json.load(dataset_file)['data']

In [41]:
adversarial_dict = {}
for qa in adversarial:
    adversarial_dict[qa['question']] = qa['attacks'][0]

In [31]:
# import json
# chatgpt_path = "/Volumes/Share/tran_s2/Frozen_Experiment/parallelQuAD/Data/Train_BatchGPT"
# adversarial_dict = {}

# count_mal = 0
# for i in range(1,100):
#     complete_path = chatgpt_path + "/{}.json".format(i)
#     with open(complete_path) as dataset_file:
#         chatgpt = json.load(dataset_file)
#     for attack in chatgpt:
#         if 'attack' in attack:
#             adversarial_dict[attack['question']] = attack
# print(len(adversarial_dict))

In [32]:
# adversarial_dict = {}
# for qas in adversarial:
#     adversarial_dict[qas['question']] = qas['attacks'][0]

In [42]:
attack_count = 0 
for qas in squad2:
    if qas['answers']['text'] == []:
        if qas['question'] in adversarial_dict:
            attack_count += 1
print(attack_count)

40450


In [43]:
# attack_data = []
# for question in squad2:
#     if question['answers']['text'] != []:
#         if question['question'] in adversarial_dict:
#             question['context'] = question['context'].strip() + \
#                 " " + adversarial_dict[question['question']]
#             attack_data.append(question)
# print(len(attack_data))

In [44]:
# import codecs
# dev_attack = {'data':attack_data}
# attack_path = "/Volumes/Share/tran_s2/Public_Datasets/SQuAD/squad-attack/dev_AR_QuAD_attack.json"
# with codecs.open(attack_path, "w", encoding='utf8') as outfile:
#     json.dump(dev_attack, outfile, ensure_ascii=False)

In [45]:
import spacy
import random
nlp = spacy.load("en_core_web_lg")

def sentence_segment(context):
    doc = nlp(context)
    sentences = [str(sent) for sent in list(doc.sents)]
    return sentences

def add_adversarial(context, adversarial):
    sentences = sentence_segment(context)
    n = len(sentences)
    adversarial_id = random.randint(0, n)
    sentences = sentences[:adversarial_id] + [adversarial] + sentences[adversarial_id:]
    new_context = " ".join(sentences)
    return new_context

def find_answer(new_context, answer_text, adversarial, original_ans_start):
    ad_index = new_context.find(adversarial)
    if original_ans_start < ad_index:
        start_find = max(0, original_ans_start - 5)
        return start_find + qas['context'][start_find:].find(answer_text)
    else:
        start_find = original_ans_start + len(adversarial_dict[qas['question']]) - 5
        return start_find + qas['context'][start_find:].find(answer_text)

In [46]:
import random
import copy
shortcut_study_dict = {6:[], 7:[], 8:[], 9:[], 10:[]}
count = {6:0, 7:0, 8:0, 9:0, 10:0}
count_num_qa = 0
for qa in squad2:
    if qa['question'] in adversarial_dict:
        if len(qa['answers']['answer_start']) == 0:
            rand = random.random()
            count_num_qa += 1
            if count_num_qa % 1000 == 0:
                print(count_num_qa)
            new_qa = copy.deepcopy(qa)
            attack = adversarial_dict[qas['question']]
            new_qa['context'] = add_adversarial(qa['context'], attack)
            
            
            if rand < 0.6:
                shortcut_study_dict[6].append(new_qa)
                count[6] += 1
            else:
                shortcut_study_dict[6].append(qa)

            if rand < 0.7:
                shortcut_study_dict[7].append(new_qa)
                count[7] += 1
            else:
                shortcut_study_dict[7].append(qa)

            if rand < 0.8:
                shortcut_study_dict[8].append(new_qa)
                count[8] += 1
            else:
                shortcut_study_dict[8].append(qa)

            if rand < 0.9:
                shortcut_study_dict[9].append(new_qa)
                count[9] += 1
            else:
                shortcut_study_dict[9].append(qa)

            shortcut_study_dict[10].append(new_qa)
            count[10] += 1

        else:
            shortcut_study_dict[6].append(qa)
            shortcut_study_dict[7].append(qa)
            shortcut_study_dict[8].append(qa)
            shortcut_study_dict[9].append(qa)
            shortcut_study_dict[10].append(qa)
    else:
        shortcut_study_dict[6].append(qa)
        shortcut_study_dict[7].append(qa)
        shortcut_study_dict[8].append(qa)
        shortcut_study_dict[9].append(qa)
        shortcut_study_dict[10].append(qa)
print(count)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
{6: 24263, 7: 28394, 8: 32449, 9: 36403, 10: 40450}


In [47]:
import codecs

shortcut_path = "/Volumes/Share/tran_s2/SQuAD2_new/data/shortcut-study/single-ans/"
for i in range(6,11):
    complete_path = shortcut_path + "{}.json".format(i)
    shortcut_data = {'data':shortcut_study_dict[i]}
    with codecs.open(complete_path, "w", encoding='utf8') as outfile:
        json.dump(shortcut_data, outfile, ensure_ascii=False)

In [12]:
import json
new_path = "/Volumes/Share/tran_s2/SQuAD2_new/data/train-squad-agent-multians.json"
with open(new_path) as dataset_file:
    squad2 = json.load(dataset_file)['data']
for qa in squad2:
    if '' in qa['answers']['text']:
        # qa['answers'] = {'answer_start': [128], 'text': ['the doctrine of the church']}
        print(qa)

In [13]:
context = "When Ibn Sina was 22 years old, he lost his father. The Samanid dynasty came to its end in December 1004. Ibn Sina seems to have declined the offers of Mahmud of Ghazni, and proceeded westwards to Urgench in modern Turkmenistan, where the vizier, regarded as a friend of scholars, gave him a small monthly stipend. The pay was small, however, so Ibn Sina wandered from place to place through the districts of Nishapur and Merv to the borders of Khorasan, seeking an opening for his talents. Qabus, the generous ruler of Tabaristan, himself a poet and a scholar, with whom Ibn Sina had expected to find asylum, was on about that date (1012) starved to death by his troops who had revolted. Ibn Sina himself was at this time stricken by a severe illness. Ibn Sina traveled to Persia after the end of the Samanid dynasty. Finally, at Gorgan, near the Caspian Sea, Ibn Sina met with a friend, who bought a dwelling near his own house in which Ibn Sina lectured on logic and astronomy. Several of Ibn Sina's treatises were written for this patron; and the commencement of his Canon of Medicine also dates from his stay in Hyrcania."
print(context[774:800])

Persia after the end of th


In [7]:
import json
import codecs
with open("/Volumes/Share/tran_s2/FinalUnanswerable/AdversarialTraining/Data/SQuAD_2.0_train_pquad_adversarial.json") as dataset_file:
    test = json.load(dataset_file)['data']
count = 0
for qas in test:

    if len(qas['answers']['text']) > 0:
        answer = qas['answers']['text'][0]
        answer_start = qas['answers']['answer_start'][0]
        context = qas['context']
        if context[answer_start:answer_start+len(answer)] != answer:
            print(qas)
            count += 1
print(count)

{'id': '573071f02461fd1900a9ce15', 'context': "There is a sizeable Greek minority of about 105,000 (disputed, sources claim higher) people, in Albania. What other countries don't boast of a Greek populace are Alexandria and Antioch. The Greek minority of Turkey, which numbered upwards of 200,000 people after the 1923 exchange, has now dwindled to a few thousand, after the 1955 Constantinople Pogrom and other state sponsored violence and discrimination. This effectively ended, though not entirely, the three-thousand-year-old presence of Hellenism in Asia Minor. There are smaller Greek minorities in the rest of the Balkan countries, the Levant and the Black Sea states, remnants of the Old Greek Diaspora (pre-19th century).", 'question': 'What other countries boast of a Greek populace ?', 'answers': {'answer_start': [38], 'text': ['105,000 (disputed, sources claim higher) people, in Albania. The Greek minority of Turkey']}}
{'id': '573084818ab72b1400f9c544', 'context': "Kievan Rus' begins

# Extract Comparing Example

In [2]:
import json
pquad_path = "/Volumes/Share/tran_s2/Frozen_Experiment/parallelQuAD/Data/PQuAD-attack/pquad-attack.json"
jia_path = "/Volumes/Share/tran_s2/FinalUnanswerable/AdversarialTraining/Data/SQuAD_2.0_train_bothAdversarial.json"
with open(pquad_path) as dataset_file:
    pquad = json.load(dataset_file)
with open(jia_path) as dataset_file:
    jia = json.load(dataset_file)['data']

In [8]:
count = 0
for question in jia:
    q = question['question']
    if question['answers']['text'] != []:
        if q in pquad:
            print(q)
            print(pquad[q])
            print(question['context'])
            print(question['answers']['text'])
            print("\n")
            count += 1
            if count == 50:
                break


What does the fluid in Neptune have a high conductivity of? 
The fluid in Uranus has a high conductivity of gas.
The mantle is equivalent to 10 to 15 Earth masses and is rich in water, ammonia and methane. As is customary in planetary science, this mixture is referred to as icy even though it is a hot, dense fluid. This fluid, which has a high electrical conductivity, is sometimes called a water–ammonia ocean. The mantle may consist of a layer of ionic water in which the water molecules break down into a soup of hydrogen and oxygen ions, and deeper down superionic water in which the oxygen crystallises but the hydrogen ions float around freely within the oxygen lattice. At a depth of 7000 km, the conditions may be such that methane decomposes into diamond crystals that rain downwards like hailstones. Very-high-pressure experiments at the Lawrence Livermore National Laboratory suggest that the base of the mantle may comprise an ocean of liquid carbon with floating solid 'diamonds'.
['el

# Debug Dataset

In [31]:
data_path = "/Volumes/Share/tran_s2/SQuAD2_new/data/shortcut-study/multi-ans/train-squad-v2-adversarial-100unanswerable.json"
import json
with open(data_path) as dataset_file:
    data = json.load(dataset_file)['data']

In [33]:
for qa in data:
    if '' in qa['answers']['text']:
        qa['answers'] = {'answer_start': [128], 'text': ['the doctrine of the church']}


In [34]:
import codecs
agent = {'data':squad2}
deeqa_dev_path = "/Volumes/Share/tran_s2/SQuAD2_new/data/shortcut-study/multi-ans/train-squad-v2-adversarial-100unanswerable.json"
with codecs.open(deeqa_dev_path, "w", encoding='utf8') as outfile:
    json.dump(agent, outfile, ensure_ascii=False)